In [ ]:
def read_images(csv_file):
    # Returns a df with the images in it
    
    import pandas as pd
    from skimage import io
    
    df = pd.read_csv(csv_file)
    read_images = []
    for file in df['image_path']:
        read_images.append(io.imread(file))

    df = df.assign(image = read_images)
    
    return df

In [ ]:
def create_hog(images):
    # Takes a list of images, returns a list of the hog descriptors
    
    from skimage.feature import hog

    hog_features = []
    for image in images:
        hog_features.append(hog(image,
                                orientations = 8,
                                pixels_per_cell = (40, 40),
                                visualize = False)
                           )
    
    return hog_features

In [1]:
def classify_images(df, x, y, random_state = 0):
    # From a df, performs image classification and returns the results
    # Takes random_state as an input to allow different reproducible results
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    
    x.append('image_path')
    train, test, y_train, y_test = train_test_split(df[x],
                                                    df[y],
                                                    test_size = 0.2,
                                                    random_state = random_state)

    if 'hog_features' in x: # stack the values of 'hog_features' to have 1 attribute / hog feature
        x.remove('hog_features')
        print(x)
        x_train = np.stack(train['hog_features'].values)
        #x_train = np.concatenate(x_train, train[x].values)
        #x_train.append(train[x])
        x_test = np.stack(test['hog_features'].values)
        #x_test.append(test[x])
        #x_test = np.concatenate(x_test, test[x].values)
        for element in x:
            x_train = np.concatenate(x_train, np.array(train[element]).values)
            x_test = np.concatenate(x_test, np.array(test[element]).values)
        #x_train = []
        #np.array(X_train)
        #x_test = []
        #for index, row in train.iterrows():
        #    x_train.append(np.concatenate([row['hog_features']], [row.drop(['image_path', 'hog_features'])]))
        #for index, row in test.iterrows():
        #    x_test.append(np.concatenate([row['hog_features']], [row.drop(['image_path', 'hog_features'])]))
    else:
        x_train = train
        x_test = test

    print('xxxxx x_train:')
    print(x_train)
    print('xxxxx y_train:')
    print(y_train)
    random_forest = RandomForestClassifier(n_estimators = 10, max_depth = 7, random_state = 0)
    random_forest.fit(x_train, y_train.values)
    predictions = random_forest.predict(x_test)
    
    return predictions, y_test, test

In [ ]:
def print_results(predictions, test):
    
    import matplotlib.pyplot as plt
    from skimage import io

    i = 0
    l = len(test)
    for index, row in test.iterrows():
        plt.subplot(1, l, i+1)
        plt.imshow(io.imread(row['image_path']))
        plt.title(str(predictions[i]))
        i = i + 1
    plt.show()